# 🧠 **02 - RAG: Sistema de Recuperação Semântica**

## 🎯 **Objetivo:**
Implementar sistema RAG completo para recuperação inteligente de conhecimento fitness.

## 📋 **O que faremos:**
1. 🔄 Carregar embeddings da etapa anterior
2. 🔍 Implementar busca semântica avançada
3. 🎯 Criar contexto personalizado por objetivo
4. ✅ Testes de recuperação de informações

---

## 1️⃣ **Carregamento dos Dados Processados**

In [1]:
# 📦 IMPORTS E SETUP

import json
import numpy as np
from pathlib import Path
from typing import Dict, List, Any, Tuple
import hashlib
import re

# Tentar carregar SentenceTransformers
try:
    from sentence_transformers import SentenceTransformer
    print("✅ SentenceTransformers disponível")
except ImportError:
    print("⚠️ SentenceTransformers não disponível")
    SentenceTransformer = None

print("📦 Imports realizados!")

✅ SentenceTransformers disponível
📦 Imports realizados!


In [2]:
# 🔄 CARREGAR DADOS DA ETAPA ANTERIOR

def carregar_dados_processados():
    """Carrega chunks e recria embeddings se necessário"""
    
    # Tentar carregar chunks salvos
    arquivo_chunks = Path("fitness_chunks.json")
    
    if arquivo_chunks.exists():
        try:
            with open(arquivo_chunks, 'r', encoding='utf-8') as f:
                dados = json.load(f)
            
            print(f"✅ Chunks carregados: {dados['total_chunks']} itens")
            return dados['chunks']
            
        except Exception as e:
            print(f"❌ Erro ao carregar chunks: {e}")
    
    # Se não conseguir carregar, processar novamente
    print("🔄 Processando base de conhecimento...")
    return reprocessar_base()

def reprocessar_base():
    """Reprocessa base de conhecimento se dados não estiverem disponíveis"""
    
    # Carregar base fitness
    arquivo_base = Path("base_conhecimento_fitness.txt")
    
    if arquivo_base.exists():
        with open(arquivo_base, 'r', encoding='utf-8') as f:
            conteudo = f.read()
    else:
        # Base mínima para demonstração
        conteudo = """
### HIPERTROFIA
Séries: 3-4
Repetições: 8-12
Descanso: 60-90 segundos
Volume alto, intensidade moderada.

### PEITO
**Supino reto**
Exercício fundamental para peitoral maior.

**Supino inclinado** 
Foca na porção superior do peitoral.
"""
    
    # Chunking básico
    chunks = []
    secoes = conteudo.split('### ')
    
    for i, secao in enumerate(secoes[1:], 1):  # Pular primeira seção vazia
        linhas = secao.strip().split('\n')
        titulo = linhas[0] if linhas else f"Seção {i}"
        texto_secao = '### ' + secao.strip()
        
        chunks.append({
            'id': len(chunks),
            'texto': texto_secao,
            'secao': titulo,
            'tokens': len(texto_secao.split()),
            'hash': hashlib.md5(texto_secao.encode()).hexdigest()[:8]
        })
    
    print(f"📄 {len(chunks)} chunks criados")
    return chunks

# Carregar dados
chunks = carregar_dados_processados()
print(f"\n📊 Total de chunks disponíveis: {len(chunks)}")

✅ Chunks carregados: 22 itens

📊 Total de chunks disponíveis: 22


## 2️⃣ **Sistema RAG Avançado**

In [3]:
# 🧠 CLASSE RAG COMPLETA

class FitnessRAGSystem:
    """Sistema RAG completo para conhecimento fitness"""
    
    def __init__(self, chunks: List[Dict]):
        self.chunks = chunks
        self.embedding_model = None
        self.embeddings = []
        
        # Tentar carregar modelo de embeddings
        if SentenceTransformer:
            self._inicializar_embeddings()
        
        # Extrair dados estruturados
        self.exercicios_por_grupo = self._extrair_exercicios()
        self.principios_por_objetivo = self._extrair_principios()
    
    def _inicializar_embeddings(self):
        """Inicializa modelo e gera embeddings"""
        try:
            print("🔄 Carregando modelo de embeddings...")
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            
            # Gerar embeddings para todos os chunks
            textos = [chunk['texto'] for chunk in self.chunks]
            self.embeddings = self.embedding_model.encode(textos)
            
            # Adicionar embeddings aos chunks
            for i, chunk in enumerate(self.chunks):
                chunk['embedding'] = self.embeddings[i]
            
            print(f"✅ Embeddings gerados: {self.embeddings.shape}")
            
        except Exception as e:
            print(f"⚠️ Erro nos embeddings: {e}")
            self.embedding_model = None
    
    def buscar_contexto(self, query: str, top_k: int = 3, threshold: float = 0.3) -> List[Dict]:
        """Busca contexto relevante usando similaridade semântica"""
        
        if self.embedding_model and self.embeddings.size > 0:
            return self._busca_semantica(query, top_k, threshold)
        else:
            return self._busca_keywords(query, top_k)
    
    def _busca_semantica(self, query: str, top_k: int, threshold: float) -> List[Dict]:
        """Busca semântica usando embeddings"""
        try:
            # Embedding da query
            query_embedding = self.embedding_model.encode([query])[0]
            
            # Calcular similaridades
            similaridades = []
            for i, chunk in enumerate(self.chunks):
                if 'embedding' in chunk:
                    sim = np.dot(query_embedding, chunk['embedding']) / (
                        np.linalg.norm(query_embedding) * np.linalg.norm(chunk['embedding'])
                    )
                    if sim >= threshold:
                        similaridades.append((sim, chunk))
            
            # Ordenar e retornar
            similaridades.sort(key=lambda x: x[0], reverse=True)
            
            resultados = []
            for sim, chunk in similaridades[:top_k]:
                resultado = chunk.copy()
                resultado['score'] = float(sim)
                resultado['metodo'] = 'semantica'
                resultados.append(resultado)
            
            return resultados
            
        except Exception as e:
            print(f"❌ Erro na busca semântica: {e}")
            return self._busca_keywords(query, top_k)
    
    def _busca_keywords(self, query: str, top_k: int) -> List[Dict]:
        """Busca por palavras-chave (fallback)"""
        query_words = set(query.lower().split())
        
        scores = []
        for chunk in self.chunks:
            chunk_words = set(chunk['texto'].lower().split())
            intersecao = len(query_words.intersection(chunk_words))
            
            if intersecao > 0:
                score = intersecao / len(query_words)
                scores.append((score, chunk))
        
        scores.sort(key=lambda x: x[0], reverse=True)
        
        resultados = []
        for score, chunk in scores[:top_k]:
            resultado = chunk.copy()
            resultado['score'] = float(score)
            resultado['metodo'] = 'keywords'
            resultados.append(resultado)
        
        return resultados
    
    def _extrair_exercicios(self) -> Dict[str, List[str]]:
        """Extrai exercícios estruturados"""
        exercicios = {
            "peito": [], "costas": [], "ombros": [],
            "bracos": [], "pernas": [], "core": []
        }
        
        for chunk in self.chunks:
            texto = chunk['texto']
            secao = chunk.get('secao', '').lower()
            
            # Detectar grupo muscular
            grupo = None
            if 'peito' in secao:
                grupo = 'peito'
            elif 'costa' in secao:
                grupo = 'costas'
            elif 'ombro' in secao:
                grupo = 'ombros'
            elif 'braço' in secao:
                grupo = 'bracos'
            elif 'perna' in secao:
                grupo = 'pernas'
            elif 'core' in secao or 'abdome' in secao:
                grupo = 'core'
            
            # Extrair exercícios (linhas com **nome**)
            if grupo and grupo in exercicios:
                linhas = texto.split('\n')
                for linha in linhas:
                    if linha.startswith('**') and linha.endswith('**'):
                        exercicio = linha.replace('**', '').strip()
                        if exercicio and exercicio not in exercicios[grupo]:
                            exercicios[grupo].append(exercicio)
        
        # Fallback se não encontrar
        if all(len(lista) == 0 for lista in exercicios.values()):
            exercicios = {
                "peito": ["Supino reto", "Supino inclinado", "Crucifixo"],
                "costas": ["Puxada frontal", "Remada curvada", "Levantamento terra"],
                "ombros": ["Desenvolvimento militar", "Elevação lateral"],
                "bracos": ["Rosca direta", "Tríceps testa"],
                "pernas": ["Agachamento", "Leg press", "Stiff"],
                "core": ["Prancha", "Abdominal"]
            }
        
        return exercicios
    
    def _extrair_principios(self) -> Dict[str, Dict]:
        """Extrai princípios de treino por objetivo"""
        principios = {
            "hipertrofia": {"series": "3-4", "repeticoes": "8-12", "descanso": "60-90s"},
            "forca": {"series": "3-5", "repeticoes": "1-6", "descanso": "2-5min"},
            "emagrecimento": {"series": "3-4", "repeticoes": "12-15+", "descanso": "30-60s"},
            "condicionamento": {"series": "3-6", "repeticoes": "15-25", "descanso": "15-45s"}
        }
        
        # Tentar extrair do texto real
        for chunk in self.chunks:
            texto = chunk['texto'].lower()
            secao = chunk.get('secao', '').lower()
            
            # Detectar objetivo
            objetivo = None
            if 'hipertrofia' in secao:
                objetivo = 'hipertrofia'
            elif 'força' in secao or 'forca' in secao:
                objetivo = 'forca'
            elif 'emagrecimento' in secao:
                objetivo = 'emagrecimento'
            elif 'condicionamento' in secao:
                objetivo = 'condicionamento'
            
            # Extrair parâmetros se encontrou objetivo
            if objetivo and objetivo in principios:
                # Buscar séries
                series_match = re.search(r'séries?:\s*(\d+-?\d*)', texto)
                if series_match:
                    principios[objetivo]['series'] = series_match.group(1)
                
                # Buscar repetições
                reps_match = re.search(r'repetições?:\s*(\d+-?\d*\+?)', texto)
                if reps_match:
                    principios[objetivo]['repeticoes'] = reps_match.group(1)
                
                # Buscar descanso
                desc_match = re.search(r'descanso:\s*([\d-]+\s*[a-z]+)', texto)
                if desc_match:
                    principios[objetivo]['descanso'] = desc_match.group(1)
        
        return principios

# Inicializar sistema RAG
print("🧠 Inicializando sistema RAG...")
rag_system = FitnessRAGSystem(chunks)

print(f"\n✅ Sistema RAG inicializado!")
print(f"📊 Exercícios disponíveis: {sum(len(ex) for ex in rag_system.exercicios_por_grupo.values())}")
print(f"🎯 Objetivos configurados: {len(rag_system.principios_por_objetivo)}")
print(f"🔍 Método de busca: {'Semântica' if rag_system.embedding_model else 'Keywords'}")

🧠 Inicializando sistema RAG...
🔄 Carregando modelo de embeddings...
✅ Embeddings gerados: (22, 384)

✅ Sistema RAG inicializado!
📊 Exercícios disponíveis: 26
🎯 Objetivos configurados: 4
🔍 Método de busca: Semântica


## 3️⃣ **Geração de Contexto Personalizado**

In [4]:
# 🎯 CONTEXTO PERSONALIZADO POR OBJETIVO

def gerar_contexto_personalizado(objetivo: str, periodicidade: int, rag: FitnessRAGSystem) -> Dict[str, Any]:
    """Gera contexto completo personalizado para o usuário"""
    
    print(f"🎯 Gerando contexto para: {objetivo} ({periodicidade}x/semana)")
    
    # Queries específicas para o objetivo
    queries = [
        f"treino {objetivo}",
        f"exercícios {objetivo}", 
        f"séries repetições {objetivo}",
        f"{periodicidade} dias semana divisão treino"
    ]
    
    # Buscar informações relevantes
    contextos_encontrados = []
    for query in queries:
        resultados = rag.buscar_contexto(query, top_k=2, threshold=0.2)
        for resultado in resultados:
            if resultado['score'] > 0.2:  # Filtro de qualidade
                contextos_encontrados.append({
                    'texto': resultado['texto'],
                    'score': resultado['score'],
                    'secao': resultado.get('secao', 'Desconhecida'),
                    'query_origem': query
                })
    
    # Remover duplicatas mantendo melhor score
    contextos_unicos = {}
    for ctx in contextos_encontrados:
        hash_texto = hashlib.md5(ctx['texto'].encode()).hexdigest()[:8]
        if hash_texto not in contextos_unicos or ctx['score'] > contextos_unicos[hash_texto]['score']:
            contextos_unicos[hash_texto] = ctx
    
    # Ordenar por relevância
    contextos_finais = sorted(contextos_unicos.values(), key=lambda x: x['score'], reverse=True)[:5]
    
    # Obter exercícios por grupo muscular
    grupos_treino = {
        2: ["peito", "costas"],
        3: ["peito", "costas", "pernas"],
        4: ["peito", "costas", "ombros", "pernas"],
        5: ["peito", "costas", "ombros", "bracos", "pernas"],
        6: ["peito", "costas", "ombros", "bracos", "pernas", "core"]
    }
    
    grupos_selecionados = grupos_treino.get(periodicidade, grupos_treino[4])
    exercicios_por_grupo = {}
    
    for grupo in grupos_selecionados:
        exercicios = rag.exercicios_por_grupo.get(grupo, [])
        exercicios_por_grupo[grupo] = exercicios
    
    # Obter princípios do objetivo
    principios = rag.principios_por_objetivo.get(objetivo, {
        "series": "3", "repeticoes": "10", "descanso": "60s"
    })
    
    # Compilar contexto final
    contexto = {
        'objetivo': objetivo,
        'periodicidade': periodicidade,
        'contextos_rag': contextos_finais,
        'exercicios_por_grupo': exercicios_por_grupo,
        'principios_treino': principios,
        'resumo_contexto': f"Contexto para {objetivo} com {len(contextos_finais)} seções relevantes",
        'total_exercicios': sum(len(ex) for ex in exercicios_por_grupo.values()),
        'grupos_musculares': list(exercicios_por_grupo.keys())
    }
    
    return contexto

print("✅ Função de contexto personalizado criada!")

✅ Função de contexto personalizado criada!


## 4️⃣ **Testes do Sistema RAG**

In [5]:
# 🧪 TESTES DO SISTEMA RAG

print("🧪 TESTANDO SISTEMA RAG COMPLETO")
print("=" * 60)

# Cenários de teste
cenarios_teste = [
    {"objetivo": "hipertrofia", "periodicidade": 4, "desc": "Ganho de massa muscular"},
    {"objetivo": "emagrecimento", "periodicidade": 5, "desc": "Perda de gordura"},
    {"objetivo": "forca", "periodicidade": 3, "desc": "Ganho de força"}
]

for i, cenario in enumerate(cenarios_teste, 1):
    print(f"\n{i}️⃣ CENÁRIO: {cenario['desc'].upper()}")
    print("-" * 40)
    
    # Gerar contexto personalizado
    contexto = gerar_contexto_personalizado(
        cenario['objetivo'], 
        cenario['periodicidade'], 
        rag_system
    )
    
    # Exibir resultados
    print(f"\n📊 RESULTADOS:")
    print(f"   • Contextos RAG encontrados: {len(contexto['contextos_rag'])}")
    print(f"   • Exercícios disponíveis: {contexto['total_exercicios']}")
    print(f"   • Grupos musculares: {', '.join(contexto['grupos_musculares'])}")
    
    print(f"\n🎯 PRINCÍPIOS DE TREINO:")
    for param, valor in contexto['principios_treino'].items():
        print(f"   • {param.title()}: {valor}")
    
    print(f"\n🔍 CONTEXTOS MAIS RELEVANTES:")
    for j, ctx in enumerate(contexto['contextos_rag'][:2], 1):
        print(f"   {j}. Score: {ctx['score']:.3f} | Seção: {ctx['secao']}")
        print(f"      Texto: {ctx['texto'][:80]}...")
    
    print(f"\n💪 EXERCÍCIOS POR GRUPO:")
    for grupo, exercicios in contexto['exercicios_por_grupo'].items():
        print(f"   • {grupo.title()}: {', '.join(exercicios[:3])}{'...' if len(exercicios) > 3 else ''}")

print("\n" + "=" * 60)
print("✅ Testes do sistema RAG concluídos!")

🧪 TESTANDO SISTEMA RAG COMPLETO

1️⃣ CENÁRIO: GANHO DE MASSA MUSCULAR
----------------------------------------
🎯 Gerando contexto para: hipertrofia (4x/semana)

📊 RESULTADOS:
   • Contextos RAG encontrados: 5
   • Exercícios disponíveis: 20
   • Grupos musculares: peito, costas, ombros, pernas

🎯 PRINCÍPIOS DE TREINO:
   • Series: 3-4
   • Repeticoes: 8-12
   • Descanso: 60-90s

🔍 CONTEXTOS MAIS RELEVANTES:
   1. Score: 0.610 | Seção: HIPERTROFIA
      Texto: ### HIPERTROFIA
- **Volume**: Alto (3-4 séries, 8-12 repetições)
- **Intensidade...
   2. Score: 0.574 | Seção: CARGA PROGRESSIVA
      Texto: ### CARGA PROGRESSIVA
- Semana 1-2: Adaptação (60-70% carga máxima estimada)
- S...

💪 EXERCÍCIOS POR GRUPO:
   • Peito: Supino reto com barra, Supino inclinado com halteres, Crucifixo inclinado...
   • Costas: Puxada frontal, Remada curvada com barra, Remada unilateral com halter...
   • Ombros: Desenvolvimento militar, Elevação lateral, Elevação posterior...
   • Pernas: Agachamento, Leg 

## 5️⃣ **Otimização e Cache**

In [6]:
# 💾 SISTEMA DE CACHE PARA OTIMIZAÇÃO

class RAGCache:
    """Cache para otimizar consultas RAG frequentes"""
    
    def __init__(self, max_size: int = 100):
        self.cache = {}
        self.max_size = max_size
        self.hits = 0
        self.misses = 0
    
    def get_cache_key(self, query: str, objetivo: str, periodicidade: int) -> str:
        """Gera chave única para cache"""
        data = f"{query}|{objetivo}|{periodicidade}"
        return hashlib.md5(data.encode()).hexdigest()[:16]
    
    def get(self, query: str, objetivo: str, periodicidade: int) -> Any:
        """Recupera do cache"""
        key = self.get_cache_key(query, objetivo, periodicidade)
        
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        
        self.misses += 1
        return None
    
    def put(self, query: str, objetivo: str, periodicidade: int, result: Any):
        """Armazena no cache"""
        key = self.get_cache_key(query, objetivo, periodicidade)
        
        # Limpar cache se muito grande
        if len(self.cache) >= self.max_size:
            # Remove item mais antigo (FIFO simples)
            oldest_key = next(iter(self.cache))
            del self.cache[oldest_key]
        
        self.cache[key] = result
    
    def stats(self) -> Dict[str, Any]:
        """Estatísticas do cache"""
        total = self.hits + self.misses
        hit_rate = (self.hits / total * 100) if total > 0 else 0
        
        return {
            'size': len(self.cache),
            'hits': self.hits,
            'misses': self.misses,
            'hit_rate': f"{hit_rate:.1f}%"
        }

# Versão otimizada da função de contexto
def gerar_contexto_com_cache(objetivo: str, periodicidade: int, rag: FitnessRAGSystem, cache: RAGCache) -> Dict[str, Any]:
    """Versão otimizada com cache"""
    
    # Tentar recuperar do cache
    cache_key = f"contexto_{objetivo}_{periodicidade}"
    resultado_cache = cache.get(cache_key, objetivo, periodicidade)
    
    if resultado_cache:
        print(f"💾 Contexto recuperado do cache")
        return resultado_cache
    
    # Gerar novo contexto
    print(f"🔄 Gerando novo contexto...")
    contexto = gerar_contexto_personalizado(objetivo, periodicidade, rag)
    
    # Salvar no cache
    cache.put(cache_key, objetivo, periodicidade, contexto)
    
    return contexto

# Inicializar cache
cache_rag = RAGCache(max_size=50)
print("✅ Sistema de cache inicializado!")

# Teste do cache
print("\n🧪 Testando cache...")

# Primeira chamada (miss)
ctx1 = gerar_contexto_com_cache("hipertrofia", 4, rag_system, cache_rag)

# Segunda chamada (hit)
ctx2 = gerar_contexto_com_cache("hipertrofia", 4, rag_system, cache_rag)

# Estatísticas
stats = cache_rag.stats()
print(f"\n📊 Estatísticas do cache:")
for key, value in stats.items():
    print(f"   • {key}: {value}")

✅ Sistema de cache inicializado!

🧪 Testando cache...
🔄 Gerando novo contexto...
🎯 Gerando contexto para: hipertrofia (4x/semana)
💾 Contexto recuperado do cache

📊 Estatísticas do cache:
   • size: 1
   • hits: 1
   • misses: 1
   • hit_rate: 50.0%


## 📊 **Resumo da Etapa 02**

### ✅ **Conquistas:**
- 🧠 Sistema RAG completo implementado
- 🔍 Busca semântica com embeddings funcionando
- 🎯 Contexto personalizado por objetivo e periodicidade
- 💾 Sistema de cache para otimização
- 📊 Extração automática de exercícios e princípios

### 🎯 **Funcionalidades Principais:**
- **Busca Semântica**: Similaridade coseno com embeddings
- **Fallback Robusto**: Busca por keywords quando necessário
- **Contexto Inteligente**: Personalizado para cada usuário
- **Cache Eficiente**: Otimização de consultas repetidas
- **Estruturas Híbridas**: Combina RAG com dados estruturados

---

🧠 **Sistema RAG fitness pronto para integração!** 🏋️‍♂️